In [3]:
from mleap.data import Data
from mleap.experiments.experiments import Experiments
from mleap.analyze_results import AnalyseResults

from mleap.shared.static_variables import EXPERIMENTS_PREDICTIONS_DIR
import numpy as np
data = Data()
from sklearn.naive_bayes import GaussianNB

In [4]:
input_io = data.open_hdf5('data/delgado.hdf5', mode='r')
output_io = data.open_hdf5('data/experiments.hdf5', mode='a')

In [3]:
analyze = AnalyseResults(hdf5_output_io=output_io, hdf5_input_io=input_io)


In [4]:
errors = analyze.calculate_loss_all_datasets(input_h5_original_datasets_group='delgado_datasets/', 
                                    output_h5_predictions_group='experiments/predictions/', 
                                    metric='mean_squared_error')

In [5]:
errors

{'LogisticRegression': array([ 0.51124003,  0.1       ,  0.        ]),
 'RandomForestClassifier': array([ 0.46773024,  0.        ,  0.        ]),
 'SVM': array([ 0.43219724,  0.        ,  0.        ])}

In [6]:
a,b = analyze.perform_t_test(errors)

In [7]:
b

,pair,t_statistic,p_value
0,LogisticRegression - RandomForestClassifier,2.92,0.18
1,LogisticRegression - SVM,2.92,0.14
2,RandomForestClassifier - SVM,2.92,0.35


In [2]:
from sklearn import datasets
iris = datasets.load_iris()
from sklearn.naive_bayes import MultinomialNB
gnb = MultinomialNB()
y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)
print("Number of mislabeled points out of a total %d points : %d" % (iris.data.shape[0],(iris.target != y_pred).sum()))

Number of mislabeled points out of a total 150 points : 7


### Neural netwroks example
https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

implementing neural networks with keras

In [20]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, Dropout
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.python.keras import optimizers 

In [21]:
def nn_deep_regressor_model():
    nn_deep_model = Sequential()
    nn_deep_model.add(Dense(288, input_dim=4, activation='relu'))
    nn_deep_model.add(Dense(144, activation='relu'))
    nn_deep_model.add(Dropout(0.5))
    nn_deep_model.add(Dense(12, activation='relu'))
    nn_deep_model.add(Dense(1))
    
    adm_optimizer  = optimizers.Adam(lr=0.001)
    nn_deep_model.compile(loss='mean_squared_error',  optimizer=adm_optimizer, metrics=['accuracy'])
    return nn_deep_model

In [22]:
nn_deep_reg = KerasRegressor(build_fn=nn_deep_regressor_model)

nEpochs = 2000
batch = iris.data.shape[0]

history_deep_reg = nn_deep_reg.fit(iris.data, 
                                   iris.target, 
                                   epochs=nEpochs, 
                                   batch_size=batch)

Epoch 1/2000
150/150 [==============================] - 2s - loss: 0.7474 - acc: 0.4067
Epoch 2/2000
150/150 [==============================] - 0s - loss: 0.5851 - acc: 0.4533
Epoch 3/2000
150/150 [==============================] - 0s - loss: 0.4920 - acc: 0.4400
Epoch 4/2000
150/150 [==============================] - 0s - loss: 0.3974 - acc: 0.5200
Epoch 5/2000
150/150 [==============================] - 0s - loss: 0.3237 - acc: 0.5600
Epoch 6/2000
150/150 [==============================] - 0s - loss: 0.3074 - acc: 0.6200
Epoch 7/2000
150/150 [==============================] - 0s - loss: 0.2703 - acc: 0.6800
Epoch 8/2000
150/150 [==============================] - 0s - loss: 0.3025 - acc: 0.6467
Epoch 9/2000
150/150 [==============================] - 0s - loss: 0.2432 - acc: 0.7267
Epoch 10/2000
150/150 [==============================] - 0s - loss: 0.1690 - acc: 0.7667
Epoch 11/2000
150/150 [==============================] - 0s - loss: 0.1750 - acc: 0.7600
Epoch 12/2000
150/150 [=======